# Note on the derivation of the reflection and transmission coefficents in
Formulation for stable and efficient implementation of the rigorous coupled-wave analysis of binary gratings
M. G. Moharam, Eric B. Grann, Drew A. Pommet, and T. K. Gaylord </br>

The final step in this paper seems straightforward but is actually non-trivial to work out to get the final working RCWA code

We start with a system of four equations:

$\begin{align}
\begin{bmatrix}
\delta_{i0}  \\ jn_Icos(\theta)\delta_{i0}
\end{bmatrix} +
\begin{bmatrix}
I  \\
-jY_I
\end{bmatrix}R &= \begin{bmatrix}
W & WX \\
V & -VX
\end{bmatrix}
\begin{bmatrix}
c^{+} \\
c^{-} \\
\end{bmatrix} \\
\begin{bmatrix}
I  \\
jY_{II} \\
\end{bmatrix}T &=
\begin{bmatrix}
WX & W \\
VX & -V \\
\end{bmatrix}
\begin{bmatrix}
c^{+} \\
c^{-} \\
\end{bmatrix}
\end{align}$

This is the original form written in the paper, but it is more transparent to write them out so you see all four equations

$
\begin{align}
\delta_{i0} + R &= Wc^{+}+WXc^{-} &(1)\\
jn_{I}cos(\theta) -jY_IR &= Vc^{+}-VXc^{-} &(2)\\
T &= WXc^{+} + Wc^{-} &(3)\\
jY_{II} &= VXc^{+} - Vc^{-} &(4)
\end{align}
$

So we have four equations and four unknowns, so we should be able to solve them. What we will do then is to solve $c^+$ and $c^-$ in terms of T using the last two equations and then substitute them out in the first two equations. Then we have a system of two equations in R and T only. Then we simplify one more time to solve R.

First we rewrite (3) for $c^{-}$

$c^{-} = W^{-1}(T-Xc^{+})$

Now we can solve $c^{+}$ using the expression (3) and (4)

$
\begin{align}
jY_{II}T &= VXC^+ -V(W^{-1}(T-WXC^+)) \\
jY_{II}T &= VXC^+ -VW^{-1}T+VW^{-1}WXC^+ \\
&=(VX+VX)c^+ -VW^{-1}T \\
2VXc^+ &= jY_{II}T + VW^{-1}T \\
c^+ &= 0.5X^{-1}V^{-1}(jY_{II}T + VW^{-1}T) \\
&= 0.5X^{-1}V^{-1}(jY_{II} + VW^{-1})T \\
&= 0.5X^{-1}(W^{-1} + jV^{-1}Y_{II})T \\
\end{align}
$

We can substitute this back into the expression for $c^{-}$ <br>
$
\begin{align}
c^{-} &= W^{-1}\bigg[T - WX\big( 0.5X^{-1}V^{-1}(jY_{II}+VW^{-1})T\big)\bigg]\\
&= W^{-1}\bigg[T - 0.5WV^{-1}(jY_{II}+VW^{-1})T\bigg] \\
&= W^{-1}T - 0.5V^{-1}(jY_{II}+VW^{-1})T \\
&= W^{-1}T - 0.5(V^{-1}jY_{II}+W^{-1})T \\
&= 0.5W^{-1}T - 0.5V^{-1}jY_{II}T \\
&= 0.5(W^{-1} -jV^{-1}Y_{II})T
\end{align}
$

#### Now we mark the steps that substitutes our expressions above into the reflection equations
First we rewrite the two reflection equations: <br>
$
\begin{align}
\delta_{i0} + R &= Wc^{+}+WXc^{-} \\
jn_{I}cos(\theta) -jY_IR &= Vc^{+}-VXc^{-}
\end{align}
$

Now we begin substitution:<br>
$
\begin{align}
\begin{matrix}
\delta_{i0} + R = W\bigg(0.5X^{-1}(W^{-1} +jV^{-1}Y_{II})T \bigg)+WX\bigg(0.5(W^{-1} -jV^{-1}Y_{II})T \bigg)  \\
jn_{I}cos(\theta) -jY_IR = V\bigg(0.5X^{-1}(W^{-1} +jV^{-1}Y_{II})T \bigg)-VX\bigg(0.5(W^{-1} -jV^{-1}Y_{II})T \bigg) \\
\vdots
\end{matrix}
\end{align}
$

## This may be able to solve, but it's not the point of this paper.
The point is to remove $X^{-1}$ which may have numerical instability problem.

BTW, the code in this repo is implemented as the paper suggested(with minor difference).

# Potential Failure Case in RCWA with Scattering Matrices

consider $a=1$ and $\lambda=1$ (all in units of microns, $L_0 = 1\times10^{-6}$ m
now consider the calculations of the eigenmodes in a linear homogeneous medium:
$
P = \begin{bmatrix} 
K_XK_y & nI-K_x^2  \\ 
K_y^2 -nI & K_xK_y\\   
\end{bmatrix}
$

where ($n=\epsilon\mu$) and $K_x = k_{inc,x}- \frac{2\pi n}{a_x}$, $K_y = k_{inc,y}- \frac{2\pi m}{a_y}$; and we consider the special case of normal incidence so $k_{inc,x} = k_{inc,y} = 0$

The E-field eigenmodes (denoted by W) is the just the identity matrix.
the eigenvalues of $\Omega^2$ are apparently just

$
P = \begin{bmatrix} 
iKz & 0  \\ 
0 & iKz\\   
\end{bmatrix}
$
where:

$
K_z = \bigg( \sqrt{k_0^2n - K_x^2 - K_y^2} \bigg)^*
$

THE ISSUE: WE CAN ENGINEER SITUATIONS WHERE KZ IS SINGULAR <br>
typically, for our gap media, we pick $n_g=1$
Additionally, we normalize by $k_0 = \frac{2\pi}{\lambda_0}$

This means that when we calculate a normalized $K_x$, we get:
$
K_x =  \frac{2\pi n}{k_0a_x} = n
$
As a result, we can see that in the $K_z$ matrix, we will have zeros on the diagonal. The main problem this has is in the calculation of the H field modes:

$
V = QW\lambda^{-1}
$

This problem is not just for gap media, but also if any of the layers or reflection/transmission regions have such properties...As a result, it isn't obvious to me that scattering matrix formalism is 'unconditionally stable'